In [79]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [80]:
df = pd.read_csv("movie_metadata.csv", encoding='utf-8')
df = df[['movie_title','genres','director_name','actor_1_name','plot_keywords']]
#df.head()

In [81]:
df = df.replace(np.nan, '', regex = True)
df['plot_keywords']= [review.replace("|"," ") for review in df['plot_keywords'].values]
df['genres']= [review.replace("|"," ") for review in df['genres'].values]

In [83]:
df['Key_words'] = ""
df['Key_words'] = df["plot_keywords"].map(str) + ' ' + df['genres'].map(str) + ' ' + df['director_name']
#print(df['Key_words'])

c = df.columns[df.dtypes == object]
df[c] = df[c].apply(lambda x: x.str.replace(r'[^\x00-\x7F]+', ''))
#df.head()

In [98]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['Key_words'])
cosine_sim = cosine_similarity(count_matrix, count_matrix)

indices = pd.Series(df.index)

def recommendations(title, cosine_sim = cosine_sim):
    recommended_movies = []
    idx = df[df['movie_title'] == title].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    top_10_indexes = list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
        
    recommend = set(df.iloc[recommended_movies]['movie_title'])
    if title in recommend:
        recommend.remove(title)
    
    return recommend

In [99]:
def main():
    print('Enter a movie: ',end='')
    movie = input()
    for item in recommendations(movie):
        print(item)

In [100]:
main()

Enter a movie: The Lord of the Rings: The Fellowship of the Ring
The Lord of the Rings: The Return of the King
The Lord of the Rings: The Two Towers
King Kong
The Hobbit: The Battle of the Five Armies
The Lovely Bones
The Hobbit: An Unexpected Journey
Krull
The Hobbit: The Desolation of Smaug
